## BenMAP data analysis

1. spatial distribution of actual values
2. spatial distribution of normalized values (by race/ethinicity population by county)
3. national sum of each endpoint by race-ethinicity
4. LA state sum of each endpoint by race-ethinictiy 
5. LA state sum of each endpoint (normalized by the corresponding population ) by race-ethinictiy


#### Note that only "Emergency Room Visits  Asthma" has ethinicity separate (for White population), so I will merge them into just "White". 
## Population data is from INMAP (2013 population with race)

In [ ]:
import geopandas as gpd
import pandas as pd
import os
import seaborn as sns
import plotly.graph_objects as go
import numpy as np
import matplotlib.pyplot as plt
import contextily as ctx
import matplotlib.colors as mcolors

# Function to subset data for a specific state or use national data
def subset_data(final_df, state_fips=None):
    if state_fips:
        # Ensure state_fips is a list
        if not isinstance(state_fips, list):
            state_fips = [state_fips]
        
        # Collect all states of interest, including neighbors
        all_fips = set(state_fips)

        # Filter the DataFrame
        return final_df[final_df['STATE_FIPS'].isin(all_fips)]
    return final_df

def plot_bar(df, x_column, y_column, title, output_dir, benmap_output, group_col=None):
    """
    Create an improved bar plot with better formatting and cleaner appearance.
    
    Parameters:
    -----------
    df : DataFrame
        The data to plot
    x_column : str
        Column name to use for the x-axis values
    y_column : str
        Column name to use for the y-axis values
    title : str
        Title for the plot
    output_dir : str
        Directory to save the output plot
    benmap_output: str
        type of benmap output(incidence or valuation)
    group_col : str, optional
        Column name to use for grouping/coloring. If None, no grouping is applied.
    """
    import matplotlib.pyplot as plt
    import seaborn as sns
    import os
    import numpy as np
    
    # Set the style
    plt.style.use('seaborn-v0_8-whitegrid')
    
    # Create figure
    fig, ax = plt.subplots(figsize=(12, 8))

    # change title
    if benmap_output == "incidence":
        plot_label = 'Health Benefits'
        if y_column == 'Mean_per_Pop':
            plot_label = plot_label + '\n (per million people) '
    elif benmap_output == 'valuation':
        if y_column == 'Mean': 
            plot_label = 'Monetized Health Benefits (in Million $)'
    
    # for valuation, make the output in million
    if benmap_output == 'valuation':
        df[y_column] = df[y_column]/1e6

    # Create the barplot with or without grouping
    if group_col:
        ax = sns.barplot(
            data=df, 
            x=x_column,
            y=y_column,
            hue=group_col,
            errorbar=None,  # Remove error bars
            palette='turbo'
        )
        add_font_size = 0 
    else:
        ax = sns.barplot(
            data=df, 
            x=x_column,
            y=y_column,
            errorbar=None,  # Remove error bars
            width=0.3,
            palette=['#3498db']  # Single color when no grouping
        )
        add_font_size= 10 # make the font size larger for none group_col case
    
    # Set title and labels with better formatting
    ax.set_title(title, fontsize=14 + add_font_size, fontweight='bold', pad=20)
    ax.set_ylabel(plot_label, fontsize=12 + add_font_size)
    ax.set_xlabel(x_column if x_column != 'Endpoint' else '', fontsize=12 + add_font_size / 2)

    # Use symlog scale for y-axis if values vary widely
    ax.set_yscale('symlog')

    # Customize legend only if group_col is provided
    if group_col:
        ax.legend(title=group_col, title_fontsize=12, fontsize=10, frameon=True)

    
    # Improve the value annotations on the bars
    for p in ax.patches:
        value = p.get_height()
        
        # Skip tiny values or exactly zero values
        if abs(value) < 0.0001:
            continue
        
        # Format numbers nicely based on their magnitude
        if abs(value) < 0.01:
            annotation = f'{value:.2f}'
        elif abs(value) < 1:
            annotation = f'{value:.1f}'
        else:
            annotation = f'{int(value)}'
        
        # Position the text better
        text_y_pos = value + (0.05 * np.sign(value))
        ax.annotate(
            annotation, 
            (p.get_x() + p.get_width() / 2., text_y_pos),
            ha='center', 
            va='bottom' if value >= 0 else 'top',
            fontsize=9 + add_font_size,
            fontweight='bold'
        )
    
    # Remove top and right spines for cleaner look
    sns.despine()
    
    # Rotate the x-tick labels for better readability
    ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right', fontsize=10 + add_font_size)

    if group_col:
        # Add subtle vertical gridlines between categories
        for i in range(len(ax.get_xticks())):
            ax.axvline(x=i - 0.5, color='grey', linestyle='--', linewidth=0.5, alpha=0.3)
    
    # Add a more prominent horizontal line at y=0
    ax.axhline(y=0, color='black', linestyle='-', linewidth=1.2)
    
    # Adjust layout to make everything fit
    plt.tight_layout()
    
    # Create filename with or without group_col info
    filename = f'{title.replace(" ", "_")}'
    if group_col:
        filename += f'_by_{group_col}'
    
    # Save with higher quality
    fig.savefig(
        os.path.join(output_dir, f'{filename}.png'),
        dpi=300,
        bbox_inches='tight'
    )
    
    #plt.show()


# Creating a pretty table with plotly
def pretty_create_table(df, columns, title):
    formatted_values = [format_values(df[col]) for col in columns]
    fig = go.Figure(data=[go.Table(
        header=dict(values=columns,
                    fill_color='paleturquoise',
                    align='left'),
        cells=dict(values=formatted_values,
                   fill_color='lavender',
                   align='left'))
    ])
    fig.update_layout(title_text=title)
    file_path = os.path.join(output_dir, f'{TARGET_GRID_LEVEL}_{title}.png')
    fig.write_image(file_path, format='png', scale=2, engine='kaleido')
    #fig.show()

# Example function to format values
def format_values(value):
    return f'{value:.4g}' if isinstance(value, (int, float)) else value

# Function to create and save a table as CSV
def create_csv(df, columns, title, output_dir):
    # Format the values in the DataFrame
    formatted_df = df[columns].applymap(format_values)
    
    # Create the file path
    file_path = os.path.join(output_dir, f'{TARGET_GRID_LEVEL}_{title}.csv')
    
    # Save the DataFrame to CSV
    formatted_df.to_csv(file_path, index=False)
    
    print(f"CSV file saved at {file_path}")


# Function to plot spatial distribution for each race on the same page using the same color scale
def plot_spatial_distribution_combined(gdf, title, y_axis, output_dir):
    unique_races = gdf['Race'].unique()
    n_races = len(unique_races)
    fig, axes = plt.subplots(1, n_races, figsize=(5 * n_races, 8), sharex=True, sharey=True)
    fig.suptitle(title)

    vmin, vmax = gdf[y_axis].min(), gdf[y_axis].max()
    max_abs = max(abs(vmin), abs(vmax)) * 0.5

    if vmax > 0 :
        vmin = 0 
        vmax = max_abs
    else:
        vmin = -max_abs
        vmax = 0

    if n_races == 1:
        axes = [axes]  # Make axes iterable if there is only one subplot

    for ax, race in zip(axes, unique_races):
        race_gdf = gdf[gdf['Race'] == race]
        race_gdf.plot(column=y_axis, cmap='viridis', vmin=vmin, vmax=vmax, legend=False, ax=ax)
        ax.set_title(f'{race}')

        # Calculate the sum of the values in the domain
        if y_axis == "Mean_per_Pop":
            total_sum = race_gdf["Mean"].sum() / race_gdf["Population"].sum() * 1000000
            # Add the sum as text within the subplot
            ax.text(0.5, -0.1, f'Sum: {total_sum:.2f}', ha='center', va='center', transform=ax.transAxes, fontsize=12, color='black')
        else:  
            total_sum = race_gdf[y_axis].sum()
            # Add the sum as text within the subplot
            ax.text(0.5, -0.1, f'Sum: {total_sum:.2f}', ha='center', va='center', transform=ax.transAxes, fontsize=12, color='black')

    # Add a single color bar
    cbar_ax = fig.add_axes([0.95, 0.15, 0.01, 0.7])
    sm = plt.cm.ScalarMappable(cmap='viridis', norm=plt.Normalize(vmin=vmin, vmax=vmax))
    sm._A = []
    fig.colorbar(sm, cax=cbar_ax)

    plt.savefig(os.path.join(output_dir, f'{y_axis}_{TARGET_GRID_LEVEL}_{title}.png'), dpi=300, bbox_inches='tight')
    #plt.show()


# Define the preferred race order
race_order = ["ALL", "BLACK", "WHITE", "ASIAN", "NATAMER", "HISPANIC"]

def plot_spatial_distribution_benmap_with_basemap(gdf, field, output_dir, region_name):


    # Ensure the GeoDataFrame is in the correct CRS for basemaps (Web Mercator)
    gdf = gdf.to_crs(epsg=3857)

    # Group by Endpoint to ensure each endpoint is plotted separately
    grouped_endpoints = gdf.groupby("Endpoint")

    for endpoint, gdf_endpoint in grouped_endpoints:
        
        # Extract available race categories within this endpoint group
        available_races = [race for race in race_order if race in gdf_endpoint["Race"].unique()]

        # change colorbar label
        if "Mortality" in endpoint:
            colorbar_label = 'Annual Avoided number of deaths'
        elif "Asthma" in endpoint:
            colorbar_label = 'Annual Avoided number of people with the symptom'
        elif "Work" in endpoint:
            colorbar_label = 'Annual Avoided work day loss'

        if field == 'Mean_per_Pop':
            colorbar_label = colorbar_label + '\n (per million) '
    
        for race in available_races:

            # Filter data for the current race within the endpoint group
            gdf_race = gdf_endpoint[gdf_endpoint["Race"] == race]

            # Ensure the field exists
            if field not in gdf_race.columns:
                #print(f'Field {field} does not exist in the data for endpoint {endpoint}, race {race}. Skipping...')
                continue

            # **Check if the field contains only zeros**
            if gdf_race[field].sum() == 0:
                #print(f'All values are zero for {endpoint}, {race}. Skipping...')
                continue  # Skip this plot

            fig, ax = plt.subplots(figsize=(10, 6))

            # Plot the spatial distribution
            vmin, vmax = gdf[field].min(), gdf[field].max()
            max_abs = max(abs(vmin), abs(vmax)) * 0.5

            if vmin >= 0 :
                vmin = 0 
                vmax = max_abs

                # Define the number of discrete bins
                num_bins = 9  # Adjust this number for more or fewer steps
                color_map = plt.cm.Reds  # Choose a sequential colormap
                bounds = np.linspace(vmin, vmax, num_bins + 1)  # Define color step boundaries
                norm = mcolors.BoundaryNorm(bounds, color_map.N)  # Create a discrete colormap
                
            else:
                vmin = -max_abs
                vmax = max_abs
                # Define the number of discrete bins
                num_bins = 18  # Adjust this number for more or fewer steps
                color_map = plt.cm.get_cmap("bwr")  # Reverse coolwarm to get cool (blue) for negative and warm (red) for positive
                bounds = np.linspace(vmin, vmax, num_bins + 1)  # Define color step boundaries
                norm = mcolors.BoundaryNorm(bounds, color_map.N)  # Create a discrete colormap


            if region_name == 'Nation':
                set_edgecolor= 'none'
            else: 
                set_edgecolor='black'
            gdf_race.plot(column=field, cmap=color_map, norm = norm, vmin=vmin, vmax=vmax, 
                          legend=False, edgecolor=set_edgecolor, ax=ax, markersize=30, alpha=0.8)

            # Add a basemap
            ctx.add_basemap(ax, source=ctx.providers.OpenStreetMap.Mapnik, zoom=6)

            ax.set_title(f'{endpoint} - {race} ({region_name})')

            # Calculate summary statistics for this race
            total_target = gdf_race[field].sum()
            max_target = gdf_race[field].max()
            min_target = gdf_race[field].min()

            # Display summary stats on the plot
            ax.text(0.5, -0.15, f'Total: {total_target:.3f}\nMax: {max_target:.3f}\nMin: {min_target:.3f}', 
                    ha='center', va='center', transform=ax.transAxes, fontsize=12, color='black')


            # Add a color bar
            sm = plt.cm.ScalarMappable(cmap=color_map, norm=norm)
            sm._A = []
            cbar_ax = fig.add_axes([0.92, 0.15, 0.02, 0.7])  
            fig.colorbar(sm, cax=cbar_ax).set_label(f'{colorbar_label}')

            # Adjust layout
            plt.subplots_adjust(left=0.1, right=0.85, top=0.9, bottom=0.2)

            # Save the figure for each endpoint-race combination
            output_filename = f'{field}_{endpoint}_{race}_{region_name}_with_basemap.png'
            output_path = os.path.join(output_dir, output_filename)
            plt.savefig(output_path, dpi=300, bbox_inches='tight')
            plt.close()

            print(f'Saved: {output_path}')



def modify_geojson(geojson_data, column):

    geojson_dict = json.loads(geojson_data)
    
    # Display properties to add to each feature
    display_properties = {
        'color': '#808080',
        'weight': 1,
        'fillOpacity': 0.75,
        'type': 'polygon',
        'iconFile': 'none',
        'layerOrder': 5,
        'onLoad': False,
        'legendEntry': 'none'
    }

    # Add displayProperties at the top level
    geojson_dict['displayProperties'] = display_properties

    if column == 'TotalPopD':
        geojson_dict['QuantityDescriptor'] = "Changes in total premature deaths"
    elif column == 'TotalPM25':
        geojson_dict['QuantityDescriptor'] = "Changes in surface PM2.5 concentrations"
    
    # Rename the quantity column to "Quantity"
    for feature in geojson_dict['features']:
        feature['properties']['Quantity'] = feature['properties'].pop(column)
    
    modified_geojson = json.dumps(geojson_dict, indent=2)
    modified_geojson = f"var INMAP_{column} = {modified_geojson};"
    
    return modified_geojson


def save_benmap_json(gdf_diff, columns_to_save, webdata_path):

    for column in columns_to_save:

        gdf_column = gdf_diff[['geometry', column]].copy()
        
        if column == 'TotalPopD':
            threshold = 0
        elif column == 'TotalPM25':
            threshold = 0.0000001

        gdf_filtered = gdf_column[(gdf_column[column].abs() > threshold)]

        geojson_data = gdf_filtered.to_json()
        modified_geojson = modify_geojson(geojson_data, column)

        filename = os.path.join(webdata_path, f'INMAP_{column}.json')
        with open(filename, 'w') as f:
            f.write(modified_geojson)

        print(f"GeoJSON data for column '{column}' has been saved to '{filename}'.")


In [ ]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

##############
# STEP 1: get BenMAP grid shapefile
##############

# Define file paths

TARGET_GRID_LEVEL = 'county'

if TARGET_GRID_LEVEL == 'county':

    grid_shapefile_path = '/Users/yunhalee/Documents/LOCAETA/RCM/BenMAP/grids/County/County.shp'
    grid_gdf = gpd.read_file(grid_shapefile_path)

elif TARGET_GRID_LEVEL == 'tracts':

    ## TODO - add BenMAP tract shapefile to get Row and Col information

    grid_shapefile_path = '/Users/yunhalee/Documents/LOCAETA/RCM/BenMAP/grids/US Census Tracts/US Census Tracts.shp'
    grid_gdf = gpd.read_file(grid_shapefile_path)

# Rename Columns
grid_gdf.rename( columns={"COL":"Col", "ROW": "Row"}, inplace=True)
# Define file paths

benmap_output_type =['incidence' , 'valuation'] # 
runnames = [ 'TN_DataCenter_NOx_2ppm','TN_DataCenter_NOx_25ppm'] #['LA_CCS', 'LA_CCS_noNH3']  # [ 'CO_CCS', 'CO_CCS_wo_NH3_VOC', 'CO_Cherokee_CCS_wo_NH3_VOC','CO_Suncor_CCS_wo_NH3_VOC', 'NEI_no_Landfill_2001411'] # # 
regions = {'TN': '47', "Nation": None} #{"CO": '08', "Nation": None}

for runname in runnames: 
    for benmap_output in benmap_output_type:

        benmap_output_file = f'/Users/yunhalee/Documents/LOCAETA/RCM/BenMAP/batchmode/APVR/control_{runname}_{TARGET_GRID_LEVEL}_inmap_2020_pm25-{benmap_output}.csv'
        #benmap_output_file = f"/Users/yunhalee/Documents/LOCAETA/RCM/BenMAP/batchmode/APVR/GUI_{benmap_output}_08052024.csv"
        output_dir = f"/Users/yunhalee/Documents/LOCAETA/LOCAETA_AQ/outputs/BenMAP/{TARGET_GRID_LEVEL}/{benmap_output}_results/{runname}"

        if not os.path.exists(output_dir):
            os.makedirs(output_dir)


        ##############
        # STEP 2: Create final dataframe containing geometry and BenMAP key results ,and compute the normalized Mean with the population (x1e6) for incidence
        ##############

        benmap = pd.read_csv(benmap_output_file)

        # this is only for GUI case
        benmap.rename( columns={"Column":"Col"}, inplace=True)
        benmap['Pollutant'] = 'PM2.5'
    
        # Merge incidence with pop_gdf on the 'Row' and 'Col' columns
        merged_df = benmap.merge(grid_gdf, on=['Row', 'Col'], how='left')

        if benmap_output == 'incidence':
            columns_to_keep = ['Endpoint', 'Pollutant', 'Author', 'Race', 'Ethnicity', 'STATE_FIPS','CNTY_FIPS', 'Row', 'Col', 'Mean', 'Population','geometry']
        else:
            columns_to_keep = ['Endpoint', 'Pollutant', 'Author', 'Race','Ethnicity', 'STATE_FIPS','CNTY_FIPS', 'Row', 'Col', 'Mean','geometry']

        final_df = merged_df[columns_to_keep]

        # change mortality endpoint
        # Modify Endpoint based on Author
        final_df.loc[final_df['Author'].str.contains('Pope', na=False), 'Endpoint'] = 'Mortality All Cause : Method 2'
        final_df.loc[final_df['Author'].str.contains('Di', na=False), 'Endpoint'] = 'Mortality All Cause : Method 1'

        # "HISPANIC" is ethinicty, not race, but I am going to reassign it as RACE for model analysis
        final_df.loc[final_df['Ethnicity'].str.contains('HISPANIC', na=False), 'Race'] = 'HISPANIC'
        final_df.drop(columns=['Ethnicity'])
        
        final_df.head()

        ##############
        # STEP 3: Create plots of total health impact (sum the health impact for each endpoint) either for State or Nation and Create map plots of local health impact
        ##############

        
        #regions = {"LA": ['22','05', "28", "48"] , "Nation": None}
        #regions = {"Nation": None}

        final_df = gpd.GeoDataFrame(final_df, geometry= "geometry")
        type(final_df)

        # Loop over the regions dictionary and process the data
        for region_name, state_fips in regions.items():
            
            # Subset the final DataFrame based on the chosen state or national
            final_df_subset = subset_data(final_df, state_fips)

            print(f"Processing data for: {region_name}")

            if benmap_output == 'incidence':
                # Group by ['Endpoint', 'Pollutant', 'Author', 'Race'] and calculate the sum of 'Mean'
                race_grouped_sum = final_df_subset.groupby(['Endpoint', 'Pollutant', 'Author', 'Race']).agg({'Mean': 'sum', "Population":"sum"}).reset_index()

                race_grouped_sum['Mean_per_Pop'] = race_grouped_sum['Mean'] / race_grouped_sum['Population'] * 1000000  # Scale by 1,000,000 for readability

                # Plotting "SUM" barplots by 'Race' for each {endpoint}, {pollutant}, {author}
                plot_bar(race_grouped_sum, "Endpoint", "Mean", f'Health benefits by Race in {region_name}', output_dir, benmap_output, 'Race')
                
                plot_bar(race_grouped_sum, "Endpoint", "Mean_per_Pop", f'Normalized Health benefits by Race in {region_name}', output_dir, benmap_output,'Race')

                # Creating a table to show the values in the barplots
                table_columns = ['Endpoint', 'Race', 'Mean', 'Mean_per_Pop']

            else:
                # Group by ['Endpoint', 'Pollutant', 'Author', 'Race'] and calculate the sum of 'Mean'
                race_grouped_sum = final_df_subset.groupby(['Endpoint', 'Pollutant', 'Author', 'Race']).agg({'Mean': 'sum'}).reset_index()

                # Plotting "SUM" barplots by 'Race' for each {endpoint}, {pollutant}, {author}
                plot_bar(race_grouped_sum, "Endpoint", "Mean", f'Monetized Health benefits by Race in {region_name}', output_dir, benmap_output, 'Race')

                # Creating a table to show the values in the barplots
                table_columns = ['Endpoint', 'Race', 'Mean']

            create_csv(race_grouped_sum, table_columns, f'Summary Table Health Benefits by Race in {region_name}', output_dir)

            #result = final_df_subset[final_df_subset['Race'] == 'ALL'].groupby('Endpoint')['Mean'].sum().reset_index()
            #plot_bar(result, "Endpoint", "Mean", f'Sum of Mean by Endpoint in {region_name}', output_dir)

            # Group by ['Endpoint', 'Pollutant', 'Author']
            grouped = final_df_subset.groupby(['Endpoint', 'Pollutant', 'Author'])

            for (endpoint, pollutant, author), group in grouped:

                #if region_name != 'Nation': # skip Nation mapping 
                    ## print(f"checking group df {group.head()}")
                    
                print("basemap mapping started")
                ## title_mean = f'Spatial Distribution for {endpoint} ({pollutant}, {author}) in {region_name}'
                ## plot_spatial_distribution_combined(group, title_mean, "Mean", output_dir)
                plot_spatial_distribution_benmap_with_basemap(group, "Mean", output_dir, region_name)
            
                if benmap_output == 'incidence':
                    group['Mean_per_Pop'] = group.apply(lambda row: row['Mean'] / row['Population'] * 1000000, axis=1)  # Scale by 1,000,000 for readability

                    plot_spatial_distribution_benmap_with_basemap(group, "Mean_per_Pop", output_dir, region_name)
                    ## title_mean_per_pop = f'Spatial Distribution for {endpoint} ({pollutant}, {author}) per Population in {region_name}'
                    ## plot_spatial_distribution_combined(group, title_mean_per_pop, "Mean_per_Pop", output_dir)

## Make health impact bar plot for all runs

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import os
import numpy as np

def plot_health_benefit(combined_df, y_label, output_dir): 
    # Convert DataFrame from wide to long format
    df_long = combined_df.melt(id_vars=["Endpoint", "Race"], var_name="Run", value_name="Value")

    # Remove rows with NaN values
    df_long = df_long.dropna(subset=["Value"])

    # Skip zero values if desired
    df_long = df_long[df_long["Value"] != 0]

    # Set Seaborn style
    sns.set(style="whitegrid")

    # Determine the number of unique "Endpoint" categories with data
    endpoints_with_data = df_long["Endpoint"].unique()
    num_endpoints = len(endpoints_with_data)

    # Adjust figure size dynamically
    fig_width = min(25, num_endpoints * 8)
    fig_height = 10

    # Create figure and axes manually for better control
    fig = plt.figure(figsize=(fig_width, fig_height))

    # Create a grid of subplots
    if num_endpoints <= 3:
        ncols = num_endpoints
    else:
        ncols = 3
        
    nrows = (num_endpoints + ncols - 1) // ncols
    gridspec = fig.add_gridspec(nrows=nrows, ncols=ncols, hspace=0.4, wspace=0.3)

    # Create each subplot individually
    for i, endpoint in enumerate(endpoints_with_data):
        row, col = i // ncols, i % ncols
        ax = fig.add_subplot(gridspec[row, col])
        
        # Filter data for this endpoint
        endpoint_data = df_long[df_long["Endpoint"] == endpoint]
        
        # Skip if no data
        if len(endpoint_data) == 0:
            continue
            
        # Create the bar plot
        ax = sns.barplot(data=endpoint_data, x="Race", y="Value", hue="Run", ax=ax)

        # Set title and labels
        ax.set_title(f"Endpoint: {endpoint}", fontsize=13, fontweight="bold")
        ax.set_xlabel("Race-Ethnicity", fontsize=15)
        ax.set_ylabel(y_label, fontsize=15)

        # Regular rotation for other endpoints
        ax.set_xticklabels(ax.get_xticklabels(), rotation=0, ha="center")
        
        # Remove legend from individual subplots (we'll add a common one later)
        ax.get_legend().remove()

        # Add vertical gridlines between categories
        # Get the actual x tick positions
        tick_positions = ax.get_xticks()
        for j in range(len(tick_positions)):
            if j > 0:  # Skip the first one to avoid a line at the left edge
                # Place gridline between tick positions
                midpoint = (tick_positions[j-1] + tick_positions[j]) / 2
                ax.axvline(x=midpoint, color='grey', linestyle='--', linewidth=0.5, alpha=0.6)
        
        # Remove legend from individual subplots (we'll add a common one later)
        if hasattr(ax, 'get_legend') and ax.get_legend() is not None:
            ax.get_legend().remove()

    # Add a single legend for the entire figure
    handles, labels = ax.get_legend_handles_labels()
    fig.legend(handles, labels, title="Run", loc="upper center", 
            bbox_to_anchor=(0.5, 0.05), ncol=min(5, len(labels)), frameon=True)

    if '\n' in y_label:
        filename_label = y_label.replace('\n', '')
    else:
        filename_label = y_label

    # Improve overall layout
    plt.tight_layout()
    plt.subplots_adjust(top=0.9)  # Make room for the legend
    plt.savefig(os.path.join(output_dir, f'{filename_label}.png'), dpi=300, bbox_inches='tight')

    plt.show()



def plot_only_mortality(combined_df, y_label, output_dir): 

    # Filter the dataframe to get only rows containing "Mortality All Cause" in the Endpoint
    mortality_df = combined_df[combined_df['Endpoint'].str.contains('Mortality All Cause') & combined_df['Race'].str.contains('ALL') ].copy()

    mortality_df.drop(columns='Race', inplace=True)
    mortality_df.set_index('Endpoint', inplace=True)
    print(mortality_df.head())

    # Create the plot
    plt.figure(figsize=(12, 8))

    # Create the bar plot
    ax = mortality_df.plot(kind='bar')

    # Customize the plot (optional)
    plt.title('')
    plt.xlabel('')
    plt.ylabel(y_label)
    plt.xticks(fontsize=12, rotation=0) # Rotate x-axis labels for better readability
    # Add grid lines for better readability
    plt.grid(axis='y', linestyle='--', alpha=0.7)

    # Add vertical gridlines between categories
    # Get the actual x tick positions
    tick_positions = ax.get_xticks()
    for j in range(len(tick_positions)):
        if j > 0:  # Skip the first one to avoid a line at the left edge
            # Place gridline between tick positions
            midpoint = (tick_positions[j-1] + tick_positions[j]) / 2
            ax.axvline(x=midpoint, color='grey', linestyle='--', linewidth=0.5, alpha=0.6)

    # Add a single legend for the entire figure
    handles, labels = ax.get_legend_handles_labels()
    plt.legend(handles, labels, title="Run", loc="upper center", 
            bbox_to_anchor=(0.5, -0.1), ncol=min(2, len(labels)), frameon=True)
        
    # Add value labels on top of bars
    for container in ax.containers:
        if 'Monetized' in y_label:
            ax.bar_label(container, fmt='{:,.0f}')
        else:
            ax.bar_label(container, fmt='{:,.1f}')

    if '\n' in y_label:
        filename_label = y_label.replace('\n', '')
    else:
        filename_label = y_label

    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, f'{filename_label} only mortality.png'), dpi=300, bbox_inches='tight')
    # Show the plot
    plt.show()

In [ ]:
import pandas as pd

# Define file paths
benmap_output_type =['incidence' , 'valuation']
TARGET_GRID_LEVEL = 'county'
runnames = [ 'TN_DataCenter_NOx_2ppm','TN_DataCenter_NOx_25ppm'] # runnames =  ['CO_CCS', 'CO_CCS_wo_NH3_VOC', 'CO_Cherokee_CCS_wo_NH3_VOC','CO_Suncor_CCS_wo_NH3_VOC' ,  'NEI_no_Landfill_2001411'] #['LA_CCS', 'LA_CCS_noNH3'] #  # ['ZERO'] # 
target_file = 'County_Summary Table Health Benefits by Race in Nation.csv'


for benmap_output in benmap_output_type:
    combined_df = None
    combined_df_normalized = None

    # Iterate over each run name
    for runname in runnames:
        # Construct the file path
        output_dir = f"/Users/yunhalee/Documents/LOCAETA/LOCAETA_AQ/outputs/BenMAP/{TARGET_GRID_LEVEL}/{benmap_output}_results/"
        
        # Read the CSV file
        df = pd.read_csv(output_dir + runname + "/" + target_file)
        
        # Filter out rows where 'Mean' is zero
        df = df[df['Mean'] != 0]
        
        if benmap_output == 'incidence' :
            # Rename columns to include the run name
            df.rename(columns={'Mean': runname, 'Mean_per_Pop': f'{runname}_(per_million)'}, inplace=True)
        else:
            df.rename(columns={'Mean': runname}, inplace=True)
        
        # Select only the necessary columns for merging
        df_mean = df[['Endpoint', 'Race', runname]]

        # Merge DataFrames on 'Endpoint' and 'Race'
        if combined_df is None:
            combined_df = df_mean  # Initialize with the first DataFrame
        else:
            combined_df = pd.merge(combined_df, df_mean, on=['Endpoint', 'Race'], how='outer')

        if benmap_output == 'incidence' :
            df_normalized = df[['Endpoint', 'Race', f'{runname}_(per_million)']]
        
            if combined_df_normalized is None:
                combined_df_normalized = df_normalized  # Initialize with the first DataFrame
            else:
                combined_df_normalized = pd.merge(combined_df_normalized, df_normalized, on=['Endpoint', 'Race'], how='outer')

    # Print or process the combined DataFrames
    print(f"Combined DataFrame for {benmap_output}:\n", combined_df)
    print(f"Combined Normalized DataFrame for {benmap_output}:\n", combined_df_normalized)

    if benmap_output == 'incidence' :
        plot_health_benefit(combined_df, 'Health Benefits', output_dir+runnames[0])
        plot_health_benefit(combined_df_normalized, "Normalized Health Benefits \n (per million)", output_dir+runnames[0])
        plot_only_mortality(combined_df, 'Health Benefits', output_dir+runnames[0])

        combined_df.to_csv(output_dir+runnames[0]+ "/Health_Benefit.csv", index=False)
        combined_df_normalized.to_csv(output_dir+runnames[0]+ "/Normalized_Health_Benefit_all_runs.csv", index=False)

    elif benmap_output == 'valuation':
        plot_health_benefit(combined_df, 'Monetized Health Benefits \n (per million $)', output_dir+runnames[0])
        plot_only_mortality(combined_df, 'Monetized Health Benefits \n (per million $)', output_dir+runnames[0])
        combined_df.to_csv(output_dir+runnames[0] + "/Monetized_Health_Benefit.csv", index=False)


In [ ]:
import pandas as pd
from great_tables import GT


GT(combined_df)

## Make JSON files from BenMAP output

In [ ]:

import json
import pandas as pd
import geopandas as gpd
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

import os

def convert_group_to_geojson(group):
    """Convert a GeoDataFrame group to a list of GeoJSON features."""
    features = json.loads(group.to_json())['features']
    for feature in features:
        # Rename 'Mean' to 'Quantity'
        feature['properties']['Quantity'] = feature['properties'].pop('Mean')
    return features

def save_grouped_benmap_json(gdf, webdata_output_dir, output_file, quantity_descriptor, threshold=1e-2):
    """Save a single JSON file with Endpoint and Race groupings."""
    
    display_properties = {
        'color': '#808080',
        'weight': 1,
        'fillOpacity': 0.75,
        'type': 'polygon',
        'iconFile': 'none',
        'layerOrder': 5,
        'onLoad': False,
        'legendEntry': 'none'
    }

    if 'Monetized' in quantity_descriptor:
        # Filter out small values
        gdf_filtered = gdf[gdf['Mean'].abs() >= 1e3]
    else: 
        # Filter out small values
        gdf_filtered = gdf[gdf['Mean'].abs() >= threshold]

    grouped = gdf_filtered.groupby(['Endpoint', 'Race'])

    print("grouped ", grouped.head())

    data_dict = {}
    for (endpoint, race), group in grouped:
        features = convert_group_to_geojson(group[['geometry', 'Mean']].copy())
        if endpoint not in data_dict:
            data_dict[endpoint] = {}

        data_dict[endpoint][race] = {
            "QuantityDescriptor": f"{quantity_descriptor} for {endpoint} in {race} group",
            "features": features
        }

    final_json = {
        "displayProperties": display_properties,
        "data": data_dict
    }

    filename = os.path.join(webdata_output_dir, f'BenMAP_{output_file}.json')

    with open(filename, 'w') as f:
        json.dump(final_json, f, indent=2)

    print(f"Grouped GeoJSON saved to '{filename}'.")



##############
# STEP 1: get BenMAP grid shapefile
##############


# Define file paths


webdata_path = '/Users/yunhalee/Documents/LOCAETA/github/LOCAETA/WebTool/Data/'


TARGET_GRID_LEVEL = 'county'

if TARGET_GRID_LEVEL == 'county':

    grid_shapefile_path = '/Users/yunhalee/Documents/LOCAETA/RCM/BenMAP/grids/County/County.shp'
    grid_gdf = gpd.read_file(grid_shapefile_path)

elif TARGET_GRID_LEVEL == 'tracts':

    ## TODO - add BenMAP tract shapefile to get Row and Col information

    grid_shapefile_path = '/Users/yunhalee/Documents/LOCAETA/RCM/BenMAP/grids/US Census Tracts/US Census Tracts.shp'
    grid_gdf = gpd.read_file(grid_shapefile_path)

# Rename Columns
grid_gdf.rename( columns={"COL":"Col", "ROW": "Row"}, inplace=True)
# Define file paths

benmap_output_type =['incidence', 'valuation'] # 
runnames = [ 'TN_DataCenter_NOx_2ppm','TN_DataCenter_NOx_25ppm'] #['LA_CCS', 'LA_CCS_noNH3']  # [ 'CO_CCS', 'CO_CCS_wo_NH3_VOC', 'CO_Cherokee_CCS_wo_NH3_VOC','CO_Suncor_CCS_wo_NH3_VOC', 'NEI_no_Landfill_2001411'] # # 
regions = {'TN': '47', "Nation": None} #{"CO": '08', "Nation": None}

for runname in runnames: 
    for benmap_output in benmap_output_type:


        webdata_output_dir = f"{webdata_path}/{runname}"
        benmap_output_file = f'/Users/yunhalee/Documents/LOCAETA/RCM/BenMAP/batchmode/APVR/control_{runname}_{TARGET_GRID_LEVEL}_inmap_2020_pm25-{benmap_output}.csv'
        #benmap_output_file = f"/Users/yunhalee/Documents/LOCAETA/RCM/BenMAP/batchmode/APVR/GUI_{benmap_output}_08052024.csv"
        output_dir = f"/Users/yunhalee/Documents/LOCAETA/LOCAETA_AQ/outputs/BenMAP/{TARGET_GRID_LEVEL}/{benmap_output}_results/{runname}"

        if not os.path.exists(output_dir):
            os.makedirs(output_dir)

        if not os.path.exists(webdata_output_dir):
            os.makedirs(webdata_output_dir)


        ##############
        # STEP 2: Create final dataframe containing geometry and BenMAP key results ,and compute the normalized Mean with the population (x1e6) for incidence
        ##############

        benmap = pd.read_csv(benmap_output_file)

        # this is only for GUI case
        benmap.rename( columns={"Column":"Col"}, inplace=True)
        benmap['Pollutant'] = 'PM2.5'
    
        # Merge incidence with pop_gdf on the 'Row' and 'Col' columns
        merged_df = benmap.merge(grid_gdf, on=['Row', 'Col'], how='left')

        if benmap_output == 'incidence':
            columns_to_keep = ['Endpoint', 'Race', 'Author', 'Ethnicity', 'Row', 'Col', 'Mean', 'geometry']
            quantity_descriptor = 'Health Benefits of'
        else:
            columns_to_keep = ['Endpoint', 'Race','Author','Ethnicity', 'Row', 'Col', 'Mean','geometry']
            quantity_descriptor = 'Monetized Health Benefits of'

        final_df = merged_df[columns_to_keep]

        # change mortality endpoint
        # Modify Endpoint based on Author
        final_df.loc[final_df['Author'].str.contains('Pope', na=False), 'Endpoint'] = 'Mortality All Cause : Method 2'
        final_df.loc[final_df['Author'].str.contains('Di', na=False), 'Endpoint'] = 'Mortality All Cause : Method 1'
        final_df.drop(columns=['Author'], inplace=True)

        # "HISPANIC" is ethinicty, not race, but I am going to reassign it as RACE for model analysis
        final_df.loc[final_df['Ethnicity'].str.contains('HISPANIC', na=False), 'Race'] = 'HISPANIC'
        final_df.drop(columns=['Ethnicity'], inplace=True)
        
        print("final df", final_df.head())

        ##############
        # STEP 3: Create plots of total health impact (sum the health impact for each endpoint) either for State or Nation and Create map plots of local health impact
        ##############

        
        #regions = {"LA": ['22','05', "28", "48"] , "Nation": None}
        #regions = {"Nation": None}

        final_gdf = gpd.GeoDataFrame(final_df, geometry= "geometry")
        type(final_gdf)

        save_grouped_benmap_json(final_gdf, webdata_output_dir, benmap_output, quantity_descriptor )

